In [2]:
%load_ext autoreload
%autoreload 2
import networkx as nx 
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
%cd ../src
tf.reset_default_graph()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/new_disk_1/luzai/NetworkCompress/src


In [4]:
from Config import * 
from Init import *
from Utils import *

def ha():
    print tf.get_default_session() 
    
with Config.sess.as_default():
    print "car",tf.get_default_session()
    ha()
    
from Net2Net import *
dbg = True
if dbg:
    config = Config(epochs=0, verbose=1, limit_data=9999)
else:
    config = Config(epochs=100, verbose=1, limit_data=1)
model_l = [["Conv2D", 'Conv2D1', {'filters': 16}],
           ["Conv2D", 'Conv2D2', {'filters': 64}],
           ["Conv2D", 'Conv2D3', {'filters': 10}],
           ['GlobalMaxPooling2D', 'GlobalMaxPooling2D1', {}],
           ['Activation', 'Activation1', {'activation_type': 'softmax'}]]
graph = MyGraph(model_l)
teacher_model = MyModel(config, graph)


car <tensorflow.python.client.session.Session object at 0x7fbe5641add0>


In [6]:
graph = tf.get_default_graph()
graph_def = graph.as_graph_def()
# i_vis_graph(graph_def)

In [7]:
teacher_model.comp_fit_eval()
teacher_model.model.save_weights("tmp.h5")
Utils.vis_model(teacher_model)

Train on 5 samples, validate on 1 samples
1/1 [==============================] - 8s

-- loss and accuracy --

[2.3503894805908203, 0.0]


In [8]:
# tf.reset_default_graph()
model_l = [["Conv2D", 'Conv2D1', {'filters': 16}],
           ["Conv2D", 'Conv2D2', {'filters': 64}],
           ["Conv2D", 'Conv2D3', {'filters': 10}],
           ['GlobalMaxPooling2D', 'GlobalMaxPooling2D1', {}],
           ['Activation', 'Activation1', {'activation_type': 'softmax'}]]
graph = MyGraph(model_l)
teacher_model = MyModel(config, graph)
net2net = Net2Net()
# student_model=MyModel(config=config,model=teacher_model)
# student_model = net2net.wider_conv2d(student_model, name='Conv2D2', width_ratio=2)
student_model = net2net.deeper_conv2d(teacher_model, name='Conv2D2')
# student_model = net2net.skip(student_model, from_name='Conv2D1', to_name='Conv2D2')

In [9]:
graph = tf.get_default_graph()
graph_def = graph.as_graph_def()
# i_vis_graph(graph_def)

In [12]:
# tf.train.export_meta_graph('tmp.pbtxt',graph_def=tf.get_default_graph().as_graph_def())
# gdef=tf.GraphDef()
# from google.protobuf import text_format 
# text_format.Merge(open('tmp.pbtxt').read(),gdef)
# i_vis_graph(gdef)
# tf.train.import_meta_graph('tmp.pbtxt')

In [10]:
minput=Input(shape=(3,32,32))
x=minput
x=Conv2D(kernel_size=3, filters=16)(x)
y=Conv2D(kernel_size=3,filters=64)(x)

In [11]:
import keras.backend.tensorflow_backend as ktf 
print ktf.int_shape(x)
print ktf.int_shape(y)

(None, 16, 30, 30)
(None, 64, 28, 28)


In [12]:
def my_resize(x,new_shape):
    import tensorflow as tf
    # original_shape = ktf.int_shape(x)
    # new_shape = tf.shape(x)
    new_shape = tf.constant(np.array(new_shape).astype('int32'))
    x = ktf.permute_dimensions(x, [0, 2, 3, 1])
    x = tf.image.resize_nearest_neighbor(x, new_shape)
    x = ktf.permute_dimensions(x, [0, 3, 1, 2])

    return  x
x = my_resize(x,(28,28))
z=keras.layers.Concatenate(axis=1)([x,y])
z=Conv2D(kernel_size=3,filters=10)(z)
model=Model(inputs=minput,outputs=z)

AttributeError: 'NoneType' object has no attribute 'inbound_nodes'